In [92]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [93]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Sunnyshore Bay")

In [94]:
# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

In [95]:
# The objective function
question_2_model.setObjective(1.0275*B[0]+1.0225*B[1]+1.0175*B[2]+1.0225*B[3]+1.0175*B[4]+1.0175*B[5], GRB.MINIMIZE)

In [96]:
# Add the balance constraints
question_2_model.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [97]:
# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model.addConstr(w[1] >= 20000, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

In [98]:
# Add the borrowing constraints
question_2_model.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model.addConstr(B[5] <= 350000, "July Borrowing Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [99]:
# Ratio constraint
question_2_model.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [100]:
# Optimally solve the problem
question_2_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x6a583230
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.906250e+04   0.000000e+00      0s
       6    1.5053662e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.505366247e+05


In [101]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model.objVal, 2))

Total Amount of Money:  150536.62


In [102]:
# Print the decision variables
print(question_2_model.printAttr('X'))


    Variable            X 
-------------------------
   Borrow[0]         5000 
   Borrow[3]      81074.9 
   Borrow[4]      61425.1 
   Wealth[0]        25000 
   Wealth[1]        27500 
   Wealth[2]        35000 
   Wealth[3]       326963 
None


In [103]:
wealth_var = question_2_model.getVarByName('Wealth[3]')
wealth_3_value = wealth_var.x
print("Wealth[3] value:", wealth_3_value)


Wealth[3] value: 326963.3753071253


In [104]:
# Print sensitivity information
print("")
print(f"Sensitivity Information for June Cash Flow Constraint {June_Cash_Flow_Constraint.pi:.2f}:")
print("(LHS, RHS, Slack): ", (question_2_model.getRow(June_Cash_Flow_Constraint).getValue(), June_Cash_Flow_Constraint.RHS, June_Cash_Flow_Constraint.slack))
print("Shadow Price: ", June_Cash_Flow_Constraint.pi)
print("Range of Feasibility: ", (June_Cash_Flow_Constraint.SARHSUp, June_Cash_Flow_Constraint.SARHSLow))


Sensitivity Information for June Cash Flow Constraint 1.02:
(LHS, RHS, Slack):  (27500.0, 27500.0, 0.0)
Shadow Price:  1.0175859950859951
Range of Feasibility:  (28846.153846153848, -0.0)


In [105]:
# Change in Objective Function Value
change_in_ofv = June_Cash_Flow_Constraint.pi * (27500 - June_Cash_Flow_Constraint.RHS)

# Additional money needed to be borrowed
additional_borrowing = -change_in_ofv  # Note: Multiply by -1 to make it positive

print("Change in Objective Function Value:", change_in_ofv)
print("Additional Money Needed to be Borrowed:", change_in_ofv + question_2_model.objVal)

Change in Objective Function Value: 0.0
Additional Money Needed to be Borrowed: 150536.6246928747


In [106]:
# Create the optimization model
question_2_model2 = gb.Model("Question 2: Sunnyshore Bay Duality")

In [107]:
y = question_2_model2.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Duality Values")

In [108]:
# The objective function
question_2_model2.setObjective(250000*y[0]+150000*y[1]+350000*y[2], GRB.MAXIMIZE)

In [109]:
# Add the constraints
question_2_model2.addConstr(1.0275*y[0] + 1.0225*y[1] + 1.0175*y[2] <= 1, "1")
question_2_model2.addConstr(1.0275*y[0] + 1.0225*y[1] <= 1, "2")
question_2_model2.addConstr(1.0275*y[0] <= 1, "3")
question_2_model2.addConstr(1.0225*y[3] + 1.0225*y[4] + 1.0175*y[5] <= 1, "4")
question_2_model2.addConstr(1.0175*y[3] + 1.0175*y[4] <= 1, "5")
question_2_model2.addConstr(1.0175*y[5] <= 1, "6")

<gurobi.Constr *Awaiting Model Update*>

In [110]:
# Optimally solve the dual problem
question_2_model2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x006898f6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+05, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4398034e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.439803440e+05


In [111]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model2.objVal, 2))

Total Amount of Money:  343980.34


In [112]:
# Create a new model
model = gb.Model("dual_model")

# Create variables
x1 = model.addVar(vtype=GRB.CONTINUOUS, name="x1")
x2 = model.addVar(vtype=GRB.CONTINUOUS, name="x2")
x3 = model.addVar(vtype=GRB.CONTINUOUS, name="x3")
x4 = model.addVar(vtype=GRB.CONTINUOUS, name="x4")
y1 = model.addVar(vtype=GRB.CONTINUOUS, name="y1")
y2 = model.addVar(vtype=GRB.CONTINUOUS, name="y2")
y3 = model.addVar(vtype=GRB.CONTINUOUS, name="y3")

# Set objective
model.setObjective(25000*x1 + 20000*x2 + 35000*x3 + 18000*x4 + 250000*y1 + 150000*y2 + 350000*y3, GRB.MAXIMIZE)

# Add constraints
model.addConstr(1.0275*x1 + 0.65*y1 <= 0)
model.addConstr(1.0225*x1 + 0.65*y1 <= 0)
model.addConstr(1.0175*x1 + 0.65*y1 <= 0)
model.addConstr(1.0225*x1 + 0.65*y2 <= 0)
model.addConstr(1.0175*x1 + 0.65*y2 <= 0)
model.addConstr(0.65*y1 <= 0)
model.addConstr(0.65*y2 <= 0)
model.addConstr(0.65*y3 <= 0)

# Optimize model
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Objective value:", model.objVal)
    print("Dual variable values:")
    print("x1 =", x1.x)
    print("x2 =", x2.x)
    print("x3 =", x3.x)
    print("x4 =", x4.x)
    print("y1 =", y1.x)
    print("y2 =", y2.x)
    print("y3 =", y3.x)
else:
    print("No solution found!")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 7 columns and 13 nonzeros
Model fingerprint: 0x264593d1
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
No solution found!


In [113]:
# Create a new model
model = gb.Model("dual")

# Define dual variables
x1 = model.addVar(lb=0, name="x1")
x2 = model.addVar(lb=0, name="x2")
x3 = model.addVar(lb=0, name="x3")
x4 = model.addVar(lb=0, name="x4")
x5 = model.addVar(lb=0, name="x5")
x6 = model.addVar(lb=0, name="x6")
x7 = model.addVar(lb=0, name="x7")
x8 = model.addVar(ub=0, lb=-GRB.INFINITY, name="x8")

# Set objective function: Maximize
model.setObjective(25000*x1 + 20000*x2 + 35000*x3 + 18000*x4 + 250000*x5 + 150000*x6 + 350000*x7, GRB.MAXIMIZE)

# Add constraints
model.addConstr(1.0275*x1 + x5 + 0.65*x8 <= 1)
model.addConstr(1.0225*x2 + x6 + 0.65*x8 <= 1)
model.addConstr(1.0175*x3 + x7 <= 1)
model.addConstr(1.0225*x4 + x7 <= 1)
model.addConstr(1.0175*x5 + x7 <= 1)
model.addConstr(1.0175*x6 + x7 <= 1)

# Optimize the model
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {model.objVal}")
    print("Dual variables:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}")
else:
    print("Optimal solution not found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 8 columns and 14 nonzeros
Model fingerprint: 0x3fea0da9
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
Optimal solution not found.


In [114]:
# Create a new model
model = gb.Model("dual")

# Define dual variables
y1 = model.addVar(name="y1")
y2 = model.addVar(name="y2")
y3 = model.addVar(name="y3")
y4 = model.addVar(name="y4")
y5 = model.addVar(lb=0, name="y5")
y6 = model.addVar(lb=0, name="y6")
y7 = model.addVar(lb=0, name="y7")
y8 = model.addVar(lb=0, name="y8")
x1 = model.addVar(lb=0, name="x1")
x2 = model.addVar(lb=0, name="x2")
x3 = model.addVar(lb=0, name="x3")
x4 = model.addVar(lb=0, name="x4")

# Set objective function: Maximize
model.setObjective(20000*y1-140000*y2+70000*y3+380000*y4+25000*x1+20000*x2+35000*x3+18000*x4+250000*y5+150000*y6+350000*y7+0*y8, GRB.MAXIMIZE)

# Add constraints
model.addConstr(-1*y1 + 1.0275*y4+y5 <=1.0275)
model.addConstr(-1*y1 + 1.0225*y3+y5 <= 1.0225)
model.addConstr(-1*y1 + 1.0175*y2+y5 <= 1.0175)
model.addConstr(-1*y2 + 1.0225*y4 + y6 <= 1.0225)
model.addConstr(-1*y2 + 1.0175*y3 + y6 <= 1.0175)
model.addConstr(-1*y3 + 1.0175*y4 + y7 <= 1.0175)

# Optimize the model
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {model.objVal}")
    print("Dual variables:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}")
else:
    print("Optimal solution not found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 12 columns and 18 nonzeros
Model fingerprint: 0x681008d7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model
Optimal solution not found.
